# 1.0 数据检查与探索

本 notebook 用于检查和探索原始 CT 数据的质量和特征。

## 目标
1. 加载并可视化 CT 数据
2. 检查数据质量（切片数、HU 范围、间距等）
3. 统计正常肺和 COPD 患者数据的分布

In [ ]:
# 导入必要的库
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from src.utils.io import load_nifti, get_nifti_info
from src.utils.data_quality import check_ct_quality, batch_quality_check, generate_quality_report

In [ ]:
# 配置路径
DATA_DIR = Path('../data/00_raw')
NORMAL_DIR = DATA_DIR / 'normal'
COPD_DIR = DATA_DIR / 'copd'

print(f"正常肺目录: {NORMAL_DIR}")
print(f"COPD 目录: {COPD_DIR}")

## 1. 数据概览

In [ ]:
# 统计文件数量
normal_files = list(NORMAL_DIR.glob('*.nii.gz')) if NORMAL_DIR.exists() else []
copd_files = list(COPD_DIR.glob('*.nii.gz')) if COPD_DIR.exists() else []

print(f"正常肺 CT 数量: {len(normal_files)}")
print(f"COPD CT 数量: {len(copd_files)}")

## 2. 单个 CT 可视化

In [ ]:
def visualize_ct_slices(ct_data, title="CT Slices", num_slices=9):
    """可视化 CT 切片"""
    fig, axes = plt.subplots(3, 3, figsize=(12, 12))
    
    # 选择均匀分布的切片
    indices = np.linspace(0, ct_data.shape[0]-1, num_slices, dtype=int)
    
    for i, (ax, idx) in enumerate(zip(axes.flat, indices)):
        ax.imshow(ct_data[idx], cmap='gray', vmin=-1000, vmax=400)
        ax.set_title(f'Slice {idx}')
        ax.axis('off')
    
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

In [ ]:
# 加载并可视化第一个 CT（如果存在）
if normal_files:
    sample_ct = load_nifti(normal_files[0])
    print(f"CT 形状: {sample_ct.shape}")
    print(f"HU 范围: [{sample_ct.min():.1f}, {sample_ct.max():.1f}]")
    visualize_ct_slices(sample_ct, title=f"Sample: {normal_files[0].name}")
else:
    print("未找到 CT 文件，请先将数据放入 data/00_raw/normal/ 目录")

## 3. 批量质量检查

In [ ]:
# 对所有文件进行质量检查
all_files = normal_files + copd_files

if all_files:
    results = {}
    for filepath in all_files:
        result = check_ct_quality(filepath)
        results[str(filepath)] = result
        status = "✓" if result.passed else "✗"
        print(f"{status} {filepath.name}: {result.messages}")
else:
    print("未找到 CT 文件")

## 4. HU 值分布分析

In [ ]:
def plot_hu_histogram(ct_data, title="HU Distribution"):
    """绘制 HU 值直方图"""
    plt.figure(figsize=(10, 4))
    
    # 只统计非背景区域
    valid_values = ct_data[ct_data > -1000]
    
    plt.hist(valid_values.flatten(), bins=100, density=True, alpha=0.7)
    plt.axvline(x=-950, color='r', linestyle='--', label='LAA-950 threshold')
    plt.xlabel('HU Value')
    plt.ylabel('Density')
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
if normal_files:
    plot_hu_histogram(sample_ct, title=f"HU Distribution: {normal_files[0].name}")

## 5. 总结

在此记录数据检查的发现和后续处理建议。

In [ ]:
# 生成质量报告
if all_files and results:
    report = generate_quality_report(results)
    print(report)